### Imports

In [1]:
import requests
import pandas as pd

### Variables

In [ ]:
# Within the API URL, filter the records to return only fyDeclared to 2013 or newer.
# US Census GDBs only go back to 2013; before that it's shapefiles only
# and I refuse to touch shapefiles, at least for the scope of this project.
api_url = r"https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries?$filter=fyDeclared ge 2013"

# These layers serve as the geometry templates for the addition of new Disaster Declarations.
# Every time the script picks up a new Disaster Declaration, it will grab the geometry of the associated
# County or Tribal Area to add to the Disaster Declaration feature class that powers the dashboard.
counties_url = r"https://services9.arcgis.com/GDVaV4SDJDDBT8gi/arcgis/rest/services/Multi_Year_US_Counties_Semi_General/FeatureServer/0"

tribal_url = r"https://services9.arcgis.com/GDVaV4SDJDDBT8gi/arcgis/rest/services/US_Tribal_Areas_Semi_Generalized/FeatureServer/0"

# This is the feature layer actually presented in the Disaster Declarations Dashboard
# All fields except geometry are taken from FEMA's Disaster Declaration Summaries;
# The geometries come from the two URLs above (Counties and Tribal Areas)
dashboard_url = r"https://services9.arcgis.com/GDVaV4SDJDDBT8gi/arcgis/rest/services/Disaster_Declarations/FeatureServer/0"


In [ ]:

response = requests.get(api_url)

data = response.json()

In [9]:

summaries = pd.DataFrame(data["DisasterDeclarationsSummaries"])

print(summaries["fyDeclared"].unique())

[2024 2017 2016 2023 2022 2021 2020 2019 2018 2025 2015 2014 2013]
